In [ ]:
# to evaluate the error
# negative error never happens
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
import os
import random
import shutil
from collections import Counter
import statistics
from numpy import linspace

DATASETS_DIR = "../tests/input_files/"
# different sets for precision and performance tests
DATASETS = ['test_100_25_15', 'test_100_25_25', 'test_100_25_50', 'test_100_25_75', 'test_100_50_15',
            'test_100_50_25', 'test_100_50_50', 'test_100_50_75']
print(DATASETS)

In [ ]:
# run and collect data
CMD_TEMPL = "../CSP {} {}"
dataset_data = {}


def get_answer(lo, hi, filename, f=None):
    """Get the smallest program answer."""
    for k_ in range(lo, hi + 1):
        cmd = CMD_TEMPL.format(filename, k_)
        if f:
            cmd += " -f"
        answer = subprocess.check_output(cmd, shell=True).decode("utf-8").split("\n")[1]
        if answer == "False":
            continue
        return k_

datasets_num = len(DATASETS)
for num, dataset in enumerate(DATASETS):
    print("Dataset {} / {} in progress".format(num + 1, datasets_num))
    dataset_metadata = dataset.split("_")
    str_len = int(dataset_metadata[1])
    str_num = int(dataset_metadata[2])
    answer = int(dataset_metadata[3])
    dataset_data[num] = {"answers": [],
                         "answers_f": [],
                         "data": {"str_len": str_len,
                                  "str_num": str_num,
                                  "answer": answer}}
    dataset_dir = os.path.join(DATASETS_DIR, dataset)
    contents = os.listdir(dataset_dir)
    for fle in contents:
        f_path = os.path.join(dataset_dir, fle)
        k = get_answer(answer, str_len, f_path)
        k_f = get_answer(answer, str_len, f_path, f=True)
        dataset_data[num]["answers"].append(k)
        dataset_data[num]["answers_f"].append(k_f)

In [ ]:
# compare answers with and without F flag
all_answers = []
all_f_answers = []
for k, v in dataset_data.items():
    all_answers.extend(v["answers"])
    all_f_answers.extend(v["answers_f"])

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(all_answers, all_f_answers, s=3)
ax.set_xlabel("Answer without F flag")
x = linspace(*ax.get_xlim())
ax.set_ylabel("Answer with F flag")
ax.plot(x, x)
ax.grid()

In [ ]:
for k, v in dataset_data.items():
    v["errors"] = [a - v["data"]["answer"] for a in v["answers"]]

In [ ]:
# visualize the error
fig = plt.figure(figsize=(15, 8))
rows = 1
cols = 8
to_plot = []
titles = []

perc_errors = []
answers = []
ax = fig.add_subplot(rows, cols, num)
for num, (k, v) in enumerate(dataset_data.items(), 1):
    # ax = fig.add_subplot(rows, cols, num)
    ans = v["data"]["answer"]
    ans_arr = [ans for _ in range(len(v["errors"]))]
    answers.extend(ans_arr)
    perc_error = [e / ans * 100 for e in v["errors"]]
    perc_errors.extend(perc_error)
    to_plot.append(perc_error)
    title = "{} {} {}".format(v["data"]["str_len"], v["data"]["str_num"], ans)
    titles.append(title)
    # ax.set_title(title)
ax = fig.add_subplot(1, 1, 1)
bp = ax.boxplot(to_plot)
ax.set_ylabel("Error")
ax.set_xlabel("Dataset")
ax.set_xticklabels(titles, rotation=45, fontsize=8)
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.scatter(answers, perc_errors)
plt.xlabel("Answer")
plt.ylabel("Error percent")
plt.grid()